# Fusing graphblas.matrix_multiply with graphblas.matrix_reduce_to_scalar

This example will go over how to use the `--graphblas-optimize` pass from `graphblas-opt` to fuse `graphblas.matrix_multiply` ops with `graphblas.matrix_reduce_to_scalar` ops into `graphblas.matrix_multiply_reduce_to_scalar` ops.

Let's first import some necessary libraries.

In [1]:
import tempfile
from mlir_graphblas.cli import GRAPHBLAS_OPT_EXE

Since [sparse tensor encodings](https://mlir.llvm.org/docs/Dialects/SparseTensorOps/#sparsetensorencodingattr) can be very verbose in MLIR, let's write some helpers to make the MLIR code more readable.

In [2]:
def tersify_mlir(input_string: str) -> str:
    terse_string = input_string
    terse_string = terse_string.replace(
        '''#sparse_tensor.encoding<{ '''
        '''dimLevelType = [ "dense", "compressed" ], '''
        '''dimOrdering = affine_map<(d0, d1) -> (d0, d1)>, '''
        '''pointerBitWidth = 64, '''
        '''indexBitWidth = 64 '''
        '''}>''', 
        "#CSR64")
    terse_string = terse_string.replace(
        '''#sparse_tensor.encoding<{ '''
        '''dimLevelType = [ "dense", "compressed" ], '''
        '''dimOrdering = affine_map<(d0, d1) -> (d1, d0)>, '''
        '''pointerBitWidth = 64, '''
        '''indexBitWidth = 64 '''
        '''}>''', 
        "#CSC64")
    return terse_string

## Simple Fusion

Here, we'll show the simplest example of how we can fuse a `graphblas.matrix_multiply` op with a `graphblas.matrix_reduce_to_scalar` op into a `graphblas.matrix_multiply_reduce_to_scalar` op.

In [3]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @fuse_adjacent(%A: tensor<?x?xf64, #CSR64>, %B: tensor<?x?xf64, #CSC64>) -> f64 {
    %C = graphblas.matrix_multiply %A, %B { semiring = "plus_plus" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64> 
    %reduce_result = graphblas.matrix_reduce_to_scalar %C { aggregator = "sum" } : tensor<?x?xf64, #CSR64> to f64
    return %reduce_result : f64
}
"""

with tempfile.NamedTemporaryFile() as temp:
    temp_file_name = temp.name
    with open(temp_file_name, 'w') as f:
        f.write(mlir_text)
    temp.flush()

    output_mlir = ! cat $temp_file_name | $GRAPHBLAS_OPT_EXE --graphblas-optimize
    output_mlir = "\n".join(output_mlir)
    output_mlir = tersify_mlir(output_mlir)

print(output_mlir)

module  {
  func @fuse_adjacent(%arg0: tensor<?x?xf64, #CSR64>, %arg1: tensor<?x?xf64, #CSC64>) -> f64 {
    %0 = graphblas.matrix_multiply_reduce_to_scalar %arg0, %arg1 {aggregator = "sum", semiring = "plus_plus"} : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to f64
    return %0 : f64
  }
}



## Simple Fusion with Mask

Fusion of `graphblas.matrix_multiply` ops with `graphblas.matrix_reduce_to_scalar` ops also works if the `graphblas.matrix_multiply` use takes a mask. 

In [4]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @fuse_adjacent_with_mask(%A: tensor<?x?xf64, #CSR64>, %B: tensor<?x?xf64, #CSC64>, %mask: tensor<?x?xf64, #CSR64>) -> f64 {
    %C = graphblas.matrix_multiply %A, %B, %mask { semiring = "plus_pair" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>, tensor<?x?xf64, #CSR64>) to tensor<?x?xf64, #CSR64> 
    %reduce_result = graphblas.matrix_reduce_to_scalar %C { aggregator = "sum" } : tensor<?x?xf64, #CSR64> to f64
    return %reduce_result : f64
}
"""

with tempfile.NamedTemporaryFile() as temp:
    temp_file_name = temp.name
    with open(temp_file_name, 'w') as f:
        f.write(mlir_text)
    temp.flush()

    output_mlir = ! cat $temp_file_name | $GRAPHBLAS_OPT_EXE --graphblas-optimize
    output_mlir = "\n".join(output_mlir)
    output_mlir = tersify_mlir(output_mlir)

print(output_mlir)

module  {
  func @fuse_adjacent_with_mask(%arg0: tensor<?x?xf64, #CSR64>, %arg1: tensor<?x?xf64, #CSC64>, %arg2: tensor<?x?xf64, #CSR64>) -> f64 {
    %0 = graphblas.matrix_multiply_reduce_to_scalar %arg0, %arg1, %arg2 {aggregator = "sum", semiring = "plus_pair"} : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>, tensor<?x?xf64, #CSR64>) to f64
    return %0 : f64
  }
}



## Non-applicable Fusion

One thing to note is that if the result of any intermediate values of the ops being fused, e.g. the result of a `graphblas.matrix_multiply`, is used elsewhere, the fusion cannot and will not apply as shown here. 

In [5]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @nofuse_multi_use(%A: tensor<?x?xf64, #CSR64>, %B: tensor<?x?xf64, #CSC64>) -> (f64, tensor<?x?xf64, #CSR64>) {
    %C = graphblas.matrix_multiply %A, %B { semiring = "plus_plus" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64> 
    %reduce_result = graphblas.matrix_reduce_to_scalar %C { aggregator = "sum" } : tensor<?x?xf64, #CSR64> to f64
    return %reduce_result, %C : f64, tensor<?x?xf64, #CSR64>
}
"""

with tempfile.NamedTemporaryFile() as temp:
    temp_file_name = temp.name
    with open(temp_file_name, 'w') as f:
        f.write(mlir_text)
    temp.flush()

    output_mlir = ! cat $temp_file_name | $GRAPHBLAS_OPT_EXE --graphblas-optimize
    output_mlir = "\n".join(output_mlir)
    output_mlir = tersify_mlir(output_mlir)

print(output_mlir)

module  {
  func @nofuse_multi_use(%arg0: tensor<?x?xf64, #CSR64>, %arg1: tensor<?x?xf64, #CSC64>) -> (f64, tensor<?x?xf64, #CSR64>) {
    %0 = graphblas.matrix_multiply %arg0, %arg1 {semiring = "plus_plus"} : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64>
    %1 = graphblas.matrix_reduce_to_scalar %0 {aggregator = "sum"} : tensor<?x?xf64, #CSR64> to f64
    return %1, %0 : f64, tensor<?x?xf64, #CSR64>
  }
}

